# Delhi Colonies Public Services Index (updated 6 Feb 2021)

## Compute the following indices:
* Index with bounding box neighbors [effective service count divided by population]
* Index with bounding box neighbors [effective service count divided by population/area]


### How to compute indices
* Load in colonies dataset (bounding box only) from Pickle **[done]**
* Merge New Population Estimates **[done]**
* Remove Rural Villages **[done]**
* Import services shapefiles **[done]**
    * Make sure correct file paths exist
    * Ensure that all shapefiles are valid using `check_shapefile` function
    * Reproject shapefiles to EPSG 7760 (if needed)
* Compute all Services indices (turn into a function) **[done]**
    * bbox neighbors, Population Size
    * bbox neighbors, Population Density

## Import modules and set constants

In [1]:
import os
import pickle
from importlib import reload
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, box
import spatial_index_utils

In [2]:
reload(spatial_index_utils)

<module 'spatial_index_utils' from 'C:\\Users\\bwbel\\Google Drive\\slum_project\\spatial_index_python\\spatial_index_utils.py'>

In [3]:
# WGS 84 / Delhi
epsg_code = 7760

## Import Colonies Dataset (bounding box only)

In [4]:
with open('colonies_bbox_nbrs25Aug2020.pkl', 'rb') as f:
    colonies_bbox_nbrs = pickle.load(f)
    
colonies_bbox_nbrs.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,area_km2,canal,railway,drain,barrier,centroid,ndmc_dist_km,population,nbrs_bbox,nbrs_dist_bbox,index
0,NEW DELHI 36,5584,None,Planned,"POLYGON Z ((1020282.788 996796.773 0.000, 1020...",1.966739,False,True,False,True,POINT (1020123.175 995898.851),5.159809,3570.060984,"[5598, 5599, 5602, 5603, 5605, 3491, 3508, 377...","[(5598, 1.074790368771482), (5599, 1.015506410...",0
1,NEW DELHI 35,5585,None,Planned,"POLYGON Z ((1019724.475 994932.797 0.000, 1019...",0.036429,False,False,False,False,POINT (1019673.024 994869.699),6.273149,323.028887,"[5586, 5594]","[(5586, 0.35712228070675794), (5594, 0.6299162...",1
2,NEW DELHI 34,5586,None,Planned,"POLYGON Z ((1019571.955 994876.019 0.000, 1019...",0.230739,False,False,False,False,POINT (1019485.484 994565.783),6.618792,2215.206473,"[5585, 5587, 5594, 5596]","[(5585, 0.35712228070675794), (5587, 0.3138054...",2
3,NEW DELHI 33,5587,None,Planned,"POLYGON Z ((1019352.702 994352.546 0.000, 1019...",0.281195,False,False,False,False,POINT (1019171.868 994576.688),6.709542,3956.166944,"[5586, 5588, 5596]","[(5586, 0.3138054943365384), (5588, 0.38643264...",3
4,NEW DELHI 32,5588,None,Planned,"POLYGON Z ((1018793.292 994224.182 0.000, 1018...",0.301253,False,False,False,False,POINT (1018785.675 994590.275),6.839299,3961.943378,"[5587, 5596, 5620, 5621]","[(5587, 0.3864326417184364), (5596, 0.62727257...",4


In [5]:
len(colonies_bbox_nbrs)

4352

## Update/Merge Population Estimates

In [6]:
# Import updated population data
updated_pop = pd.read_csv("pop_settlement_22dec2020.csv")
updated_pop.head()

,population,area,uso_area_u,uso_final
0,3570.061035,NEW DELHI 36,5584,Planned
1,320.568634,NEW DELHI 35,5585,Planned
2,2215.206543,NEW DELHI 34,5586,Planned
3,3956.166992,NEW DELHI 33,5587,Planned
4,3961.943359,NEW DELHI 32,5588,Planned


In [7]:
# rename population column to make distinct in upcoming merge
updated_pop = updated_pop.rename(columns={"population":"population_new"})
updated_pop.head()

,population_new,area,uso_area_u,uso_final
0,3570.061035,NEW DELHI 36,5584,Planned
1,320.568634,NEW DELHI 35,5585,Planned
2,2215.206543,NEW DELHI 34,5586,Planned
3,3956.166992,NEW DELHI 33,5587,Planned
4,3961.943359,NEW DELHI 32,5588,Planned


In [8]:
# Restrict dataframe to only two columns:
# layer: population data
# uso_area_u: unique id for colonies
updated_pop = updated_pop[['population_new', 'uso_area_u']]
updated_pop.head()

,population_new,uso_area_u
0,3570.061035,5584
1,320.568634,5585
2,2215.206543,5586
3,3956.166992,5587
4,3961.943359,5588


In [9]:
# Left merge updated population data with colonies data
# Note that colonies_bbox_nbrs has 15 more colonies than dataset with updated population estimates
colonies_bbox_nbrs = colonies_bbox_nbrs.merge(updated_pop, how='left', 
                          left_on="USO_AREA_U", right_on='uso_area_u')
colonies_bbox_nbrs.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,area_km2,canal,railway,drain,barrier,centroid,ndmc_dist_km,population,nbrs_bbox,nbrs_dist_bbox,index,population_new,uso_area_u
0,NEW DELHI 36,5584,None,Planned,"POLYGON Z ((1020282.788 996796.773 0.000, 1020...",1.966739,False,True,False,True,POINT (1020123.175 995898.851),5.159809,3570.060984,"[5598, 5599, 5602, 5603, 5605, 3491, 3508, 377...","[(5598, 1.074790368771482), (5599, 1.015506410...",0,3570.061035,5584.0
1,NEW DELHI 35,5585,None,Planned,"POLYGON Z ((1019724.475 994932.797 0.000, 1019...",0.036429,False,False,False,False,POINT (1019673.024 994869.699),6.273149,323.028887,"[5586, 5594]","[(5586, 0.35712228070675794), (5594, 0.6299162...",1,320.568634,5585.0
2,NEW DELHI 34,5586,None,Planned,"POLYGON Z ((1019571.955 994876.019 0.000, 1019...",0.230739,False,False,False,False,POINT (1019485.484 994565.783),6.618792,2215.206473,"[5585, 5587, 5594, 5596]","[(5585, 0.35712228070675794), (5587, 0.3138054...",2,2215.206543,5586.0
3,NEW DELHI 33,5587,None,Planned,"POLYGON Z ((1019352.702 994352.546 0.000, 1019...",0.281195,False,False,False,False,POINT (1019171.868 994576.688),6.709542,3956.166944,"[5586, 5588, 5596]","[(5586, 0.3138054943365384), (5588, 0.38643264...",3,3956.166992,5587.0
4,NEW DELHI 32,5588,None,Planned,"POLYGON Z ((1018793.292 994224.182 0.000, 1018...",0.301253,False,False,False,False,POINT (1018785.675 994590.275),6.839299,3961.943378,"[5587, 5596, 5620, 5621]","[(5587, 0.3864326417184364), (5596, 0.62727257...",4,3961.943359,5588.0


In [10]:
# Identify and save colonies with missing data for updated population estimates
#colonies_bbox_nbrs[colonies_bbox_nbrs['population_new'].isna()].to_csv("missing_colonies.csv")

In [11]:
# Create new column that replaces colonies with missing updated population estimates with older estimates
colonies_bbox_nbrs['population_updated'] = colonies_bbox_nbrs['population_new'].fillna(colonies_bbox_nbrs['population'])

In [12]:
colonies_bbox_nbrs.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,area_km2,canal,railway,drain,barrier,centroid,ndmc_dist_km,population,nbrs_bbox,nbrs_dist_bbox,index,population_new,uso_area_u,population_updated
0,NEW DELHI 36,5584,None,Planned,"POLYGON Z ((1020282.788 996796.773 0.000, 1020...",1.966739,False,True,False,True,POINT (1020123.175 995898.851),5.159809,3570.060984,"[5598, 5599, 5602, 5603, 5605, 3491, 3508, 377...","[(5598, 1.074790368771482), (5599, 1.015506410...",0,3570.061035,5584.0,3570.061035
1,NEW DELHI 35,5585,None,Planned,"POLYGON Z ((1019724.475 994932.797 0.000, 1019...",0.036429,False,False,False,False,POINT (1019673.024 994869.699),6.273149,323.028887,"[5586, 5594]","[(5586, 0.35712228070675794), (5594, 0.6299162...",1,320.568634,5585.0,320.568634
2,NEW DELHI 34,5586,None,Planned,"POLYGON Z ((1019571.955 994876.019 0.000, 1019...",0.230739,False,False,False,False,POINT (1019485.484 994565.783),6.618792,2215.206473,"[5585, 5587, 5594, 5596]","[(5585, 0.35712228070675794), (5587, 0.3138054...",2,2215.206543,5586.0,2215.206543
3,NEW DELHI 33,5587,None,Planned,"POLYGON Z ((1019352.702 994352.546 0.000, 1019...",0.281195,False,False,False,False,POINT (1019171.868 994576.688),6.709542,3956.166944,"[5586, 5588, 5596]","[(5586, 0.3138054943365384), (5588, 0.38643264...",3,3956.166992,5587.0,3956.166992
4,NEW DELHI 32,5588,None,Planned,"POLYGON Z ((1018793.292 994224.182 0.000, 1018...",0.301253,False,False,False,False,POINT (1018785.675 994590.275),6.839299,3961.943378,"[5587, 5596, 5620, 5621]","[(5587, 0.3864326417184364), (5596, 0.62727257...",4,3961.943359,5588.0,3961.943359


In [13]:
# Remove extraneous columns
colonies_bbox_nbrs = colonies_bbox_nbrs.drop(columns=['uso_area_u', 'population', 'population_new'])

# Rename 'population_new' column as 'population'
colonies_bbox_nbrs = colonies_bbox_nbrs.rename(columns={'population_updated': 'population'})

colonies_bbox_nbrs.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,area_km2,canal,railway,drain,barrier,centroid,ndmc_dist_km,nbrs_bbox,nbrs_dist_bbox,index,population
0,NEW DELHI 36,5584,None,Planned,"POLYGON Z ((1020282.788 996796.773 0.000, 1020...",1.966739,False,True,False,True,POINT (1020123.175 995898.851),5.159809,"[5598, 5599, 5602, 5603, 5605, 3491, 3508, 377...","[(5598, 1.074790368771482), (5599, 1.015506410...",0,3570.061035
1,NEW DELHI 35,5585,None,Planned,"POLYGON Z ((1019724.475 994932.797 0.000, 1019...",0.036429,False,False,False,False,POINT (1019673.024 994869.699),6.273149,"[5586, 5594]","[(5586, 0.35712228070675794), (5594, 0.6299162...",1,320.568634
2,NEW DELHI 34,5586,None,Planned,"POLYGON Z ((1019571.955 994876.019 0.000, 1019...",0.230739,False,False,False,False,POINT (1019485.484 994565.783),6.618792,"[5585, 5587, 5594, 5596]","[(5585, 0.35712228070675794), (5587, 0.3138054...",2,2215.206543
3,NEW DELHI 33,5587,None,Planned,"POLYGON Z ((1019352.702 994352.546 0.000, 1019...",0.281195,False,False,False,False,POINT (1019171.868 994576.688),6.709542,"[5586, 5588, 5596]","[(5586, 0.3138054943365384), (5588, 0.38643264...",3,3956.166992
4,NEW DELHI 32,5588,None,Planned,"POLYGON Z ((1018793.292 994224.182 0.000, 1018...",0.301253,False,False,False,False,POINT (1018785.675 994590.275),6.839299,"[5587, 5596, 5620, 5621]","[(5587, 0.3864326417184364), (5596, 0.62727257...",4,3961.943359


In [14]:
# Check that we have same number of colonies
len(colonies_bbox_nbrs)

4352

In [15]:
# Confirm no population estimates are missing
sum(colonies_bbox_nbrs['population'].isna())

0

## Remove "Rural Villages"

In [16]:
colonies_bbox_nbrs = colonies_bbox_nbrs[colonies_bbox_nbrs['USO_FINAL'] != 'RV']

In [17]:
# Check that number of colonies
len(colonies_bbox_nbrs)

4141

## Import services shapefiles

In [18]:
# Define filepaths

services_dir = os.path.join('shapefiles', 'Spatial_Index_GIS', 'Public Services')

bank_fp = os.path.join(services_dir, 'Banking', 'Banking.shp')
health_fp = os.path.join(services_dir, 'Health', 'Health.shp')
road_fp = os.path.join(services_dir, 'Major Road', 'Road.shp')
police_fp = os.path.join(services_dir, 'Police', 'Police Station.shp')
ration_fp = os.path.join(services_dir, 'Ration', 'Ration.shp')
school_fp = os.path.join(services_dir, 'School', 'schools7760.shp')
transport_fp = os.path.join(services_dir, 'Transport', 'Transport.shp')

# boundary of Delhi
delhi_bounds_filepath = os.path.join('shapefiles', 'delhi_bounds_buffer.shp')

# Check that all filepaths exist
filepath_list = [bank_fp, health_fp, road_fp, police_fp, ration_fp, school_fp, transport_fp, delhi_bounds_filepath]

for filepath in filepath_list:
    if not os.path.exists(filepath):
        print('{} does not exist'.format(filepath))

In [19]:
# Import services
bank = gpd.read_file(bank_fp)
health = gpd.read_file(health_fp)
road = gpd.read_file(road_fp)
police = gpd.read_file(police_fp)
ration = gpd.read_file(ration_fp)
school = gpd.read_file(school_fp)
transport = gpd.read_file(transport_fp)

## Check validity of services shapefiles
* Duplicate rows are okay for ATMs (I assume that ATM locations for the same bank in a similar location will seem to be counted twice)
* Look specifically for invalid geometries and whether shapefile is fully contained within Delhi

In [20]:
spatial_index_utils.check_shapefile(gdf=bank, gdf_name='bank', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

bank has duplicate rows: True
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in bank are of type Point: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [bank_name, Latitude, Longitude, Type, geometry, geom_type]
Index: []
----------------------------------------------------
bank shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


In [21]:
spatial_index_utils.check_shapefile(gdf=health, gdf_name='health', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

health has duplicate rows: False
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in health are of type Point: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [Hospital_C, ADDRESS, X, Y, geometry, geom_type]
Index: []
----------------------------------------------------
health shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


In [22]:
spatial_index_utils.check_shapefile(gdf=road, gdf_name='road', 
                                    geom_type='Line', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

road has duplicate rows: False
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in road are of type Line: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [FID, RD_NM, RD_CLS, RD_LANES, RD_TP_SRF, RD_MB, RD_ONEWAY, EL_GND, DIST_NM, ONEWAY, Speed_kmph, geometry, geom_type]
Index: []
----------------------------------------------------
road shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


In [23]:
spatial_index_utils.check_shapefile(gdf=police, gdf_name='police', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

police has duplicate rows: False
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in police are of type Point: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [NAME, POLICE_STA, DISTRICT, x, y, geometry, geom_type]
Index: []
----------------------------------------------------
police shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


In [24]:
spatial_index_utils.check_shapefile(gdf=ration, gdf_name='ration', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

ration has duplicate rows: False
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in ration are of type Point: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [S No., License No, FPS ID, Circle, FPS Shop N, Address Of, Latitude, Longitude, Source, geometry, geom_type]
Index: []
----------------------------------------------------
ration shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


In [25]:
spatial_index_utils.check_shapefile(gdf=school, gdf_name='school', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

school has duplicate rows: False
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in school are of type Point: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [objectid_1, objectid, vilname, schname, schcd, schcat, school_cat, pincode, rururb, location, schtype, school_typ, schmgt, management, dtname, stname, stcode11, dtcode11, sdtcode11, sdtname, geometry, geom_type]
Index: []

[0 rows x 22 columns]
----------------------------------------------------
school shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


In [26]:
spatial_index_utils.check_shapefile(gdf=transport, gdf_name='transport', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

transport has duplicate rows: False
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in transport are of type Point: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [stop_id, stop_name, stop_lat, stop_lon, Type, geometry, geom_type]
Index: []
----------------------------------------------------
transport shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


## Check CRS (all shapefiles should be in EPSG: 7760)

In [27]:
bank.crs == health.crs == road.crs == police.crs == ration.crs == school.crs == transport.crs

True

In [28]:
bank.crs

<Projected CRS: EPSG:7760>
Name: WGS 84 / Delhi
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: India - Delhi
- bounds: (76.83, 28.4, 77.34, 28.89)
Coordinate Operation:
- name: Delhi NSF LCC
- method: Lambert Conic Conformal (2SP)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [29]:
colonies_bbox_nbrs.crs == bank.crs

True

## Define Point and Line Services

In [30]:
# Define all point services as dictionary
# makes it easier to calculate all point
# services with one function
point_services = {'bank': bank,
                  'health': health,
                  'police': police,
                  'ration': ration,
                  'school': school,
                  'transport': transport}

line_services = {'road': road}

## Calculate all service indices in one function

In [31]:
from spatial_index_utils import calc_all_services

### Calculate PSI for bbox neighbors using Population Size

In [32]:
colonies_bbox_psi_popsize = calc_all_services(polygon_gdf = colonies_bbox_nbrs, 
                                       point_services = point_services, 
                                       line_services = line_services, 
                                       epsg_code = epsg_code, 
                                       pcen_denom = "pop",
                                       nbr_dist_colname = 'nbrs_dist_bbox')

colonies_bbox_psi_popsize = colonies_bbox_psi_popsize.rename(columns={'road_count':'road_length'})
colonies_bbox_psi_popsize.head()

GeoDataFrame now has the following CRS:

epsg:7760
bank service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
health service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
police service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
ration service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
school service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
transport service index is completed
--------------------------------------------------------
all point services completed
GeoDataFrame now has the following CRS:

epsg:7760
road service is completed


,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,area_km2,canal,railway,drain,barrier,...,school_pcen,school_idx,transport_count,transport_pcen,transport_idx,road_length,road_pcen,road_idx,unnorm_psi,norm_psi
0,NEW DELHI 36,5584,None,Planned,"POLYGON Z ((1020282.788 996796.773 0.000, 1020...",1.966739,False,True,False,True,...,0.000719,0.001912,3,0.006834,0.012300,1.951928,0.003171,0.014723,0.006122,0.008270
1,NEW DELHI 35,5585,None,Planned,"POLYGON Z ((1019724.475 994932.797 0.000, 1019...",0.036429,False,False,False,False,...,0.002299,0.006113,0,0.015705,0.028266,0.000000,0.001126,0.005229,0.029550,0.039919
2,NEW DELHI 34,5586,None,Planned,"POLYGON Z ((1019571.955 994876.019 0.000, 1019...",0.230739,False,False,False,False,...,0.003051,0.008112,6,0.004094,0.007369,0.000000,0.000681,0.003162,0.007720,0.010429
3,NEW DELHI 33,5587,None,Planned,"POLYGON Z ((1019352.702 994352.546 0.000, 1019...",0.281195,False,False,False,False,...,0.002216,0.005893,0,0.002169,0.003903,0.000000,0.000437,0.002030,0.003928,0.005306
4,NEW DELHI 32,5588,None,Planned,"POLYGON Z ((1018793.292 994224.182 0.000, 1018...",0.301253,False,False,False,False,...,0.002003,0.005326,2,0.001473,0.002651,0.747875,0.000650,0.003016,0.001916,0.002588


### Calculate PSI for bbox neighbors using Population Density

In [33]:
colonies_bbox_psi_popdensity = calc_all_services(polygon_gdf = colonies_bbox_nbrs, 
                                       point_services = point_services, 
                                       line_services = line_services, 
                                       epsg_code = epsg_code, 
                                       pcen_denom = "popdensity",
                                       nbr_dist_colname = 'nbrs_dist_bbox')

colonies_bbox_psi_popdensity = colonies_bbox_psi_popdensity.rename(columns={'road_count':'road_length'})
colonies_bbox_psi_popdensity.head()

GeoDataFrame now has the following CRS:

epsg:7760
bank service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
health service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
police service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
ration service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
school service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
transport service index is completed
--------------------------------------------------------
all point services completed
GeoDataFrame now has the following CRS:

epsg:7760
road service is completed


,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,area_km2,canal,railway,drain,barrier,...,school_pcen,school_idx,transport_count,transport_pcen,transport_idx,road_length,road_pcen,road_idx,unnorm_psi,norm_psi
0,NEW DELHI 36,5584,None,Planned,"POLYGON Z ((1020282.788 996796.773 0.000, 1020...",1.966739,False,True,False,True,...,0.001414,0.188376,3,0.013441,0.411841,1.951928,0.006236,0.589290,0.276551,0.403276
1,NEW DELHI 35,5585,None,Planned,"POLYGON Z ((1019724.475 994932.797 0.000, 1019...",0.036429,False,False,False,False,...,0.000084,0.011156,0,0.000572,0.017531,0.000000,0.000041,0.003877,0.035676,0.052024
2,NEW DELHI 34,5586,None,Planned,"POLYGON Z ((1019571.955 994876.019 0.000, 1019...",0.230739,False,False,False,False,...,0.000704,0.093782,6,0.000945,0.028948,0.000000,0.000157,0.014846,0.056871,0.082931
3,NEW DELHI 33,5587,None,Planned,"POLYGON Z ((1019352.702 994352.546 0.000, 1019...",0.281195,False,False,False,False,...,0.000623,0.083025,0,0.000610,0.018685,0.000000,0.000123,0.011620,0.034029,0.049623
4,NEW DELHI 32,5588,None,Planned,"POLYGON Z ((1018793.292 994224.182 0.000, 1018...",0.301253,False,False,False,False,...,0.000603,0.080385,2,0.000444,0.013598,0.747875,0.000196,0.018493,0.019635,0.028633


## Inspecting PSI results

In [35]:
count_cols = [colname for colname in colonies_bbox_psi_popdensity.columns if colname.endswith('_count')]
for count_col in count_cols:
    print('There are', len(colonies_bbox_psi_popdensity[colonies_bbox_psi_popdensity[count_col] < 0]), 'negative values in', count_col, 'column')

There are 0 negative values in bank_count column
There are 0 negative values in health_count column
There are 0 negative values in police_count column
There are 0 negative values in ration_count column
There are 0 negative values in school_count column
There are 0 negative values in transport_count column


In [36]:
pcen_cols = [colname for colname in colonies_bbox_psi_popdensity.columns if colname.endswith('_pcen')]
for pcen_col in pcen_cols:
    print('There are', len(colonies_bbox_psi_popdensity[colonies_bbox_psi_popdensity[pcen_col] < 0]), 'negative values in', pcen_col, 'column')

There are 0 negative values in bank_pcen column
There are 0 negative values in health_pcen column
There are 0 negative values in police_pcen column
There are 0 negative values in ration_pcen column
There are 0 negative values in school_pcen column
There are 0 negative values in transport_pcen column
There are 0 negative values in road_pcen column


In [39]:
idx_cols = [colname for colname in colonies_bbox_psi_popdensity.columns if colname.endswith('_idx')]
for idx_col in idx_cols:
    print('There are', len(colonies_bbox_psi_popdensity[colonies_bbox_psi_popdensity[idx_col] < 0]), 'negative values in', idx_col, 'column')

There are 0 negative values in bank_idx column
There are 0 negative values in health_idx column
There are 0 negative values in police_idx column
There are 0 negative values in ration_idx column
There are 0 negative values in school_idx column
There are 0 negative values in transport_idx column
There are 0 negative values in road_idx column


In [40]:
colonies_bbox_psi_popdensity['bank_idx'].describe()

count    4141.000000
mean        0.013349
std         0.048851
min         0.000000
25%         0.000250
50%         0.002280
75%         0.010503
max         1.000000
Name: bank_idx, dtype: float64

In [44]:
colonies_bbox_psi_popdensity['unnorm_psi'].describe()

count    4141.000000
mean        0.015870
std         0.038686
min         0.000000
25%         0.000760
50%         0.005144
75%         0.017190
max         0.685761
Name: unnorm_psi, dtype: float64

In [43]:
colonies_bbox_psi_popdensity['norm_psi'].describe()

count    4141.000000
mean        0.023142
std         0.056413
min         0.000000
25%         0.001108
50%         0.007501
75%         0.025067
max         1.000000
Name: norm_psi, dtype: float64

## Save Files

In [45]:
bbox_drop_columns = ['nbrs_bbox', 'nbrs_dist_bbox', 'centroid']

In [46]:
colonies_bbox_psi_popsize.drop(columns=bbox_drop_columns).to_file('delhi_psi_bbox_popsize_norv_6Feb2021.shp')
colonies_bbox_psi_popdensity.drop(columns=bbox_drop_columns).to_file('delhi_psi_bbox_popdensity_norv_6Feb2021.shp')

In [47]:
colonies_bbox_psi_popsize.to_csv('delhi_psi_bbox_popsize_norv_6Feb2021.csv')
colonies_bbox_psi_popdensity.to_csv('delhi_psi_bbox_popdensity_norv_6Feb2021.csv')

In [48]:
with open('colonies_bbox_psi_popsize_norv_6Feb2021.pkl', 'wb') as f:
    pickle.dump(colonies_bbox_psi_popsize, f)
    
with open('colonies_bbox_psi_popdensity_norv_6Feb2021.pkl', 'wb') as f:
    pickle.dump(colonies_bbox_psi_popdensity, f)